In [1]:
import pandas as pd
from datetime import date

In [2]:
DATOS = pd.read_csv('./INPUT/DEFINITIVAS_8-06.csv', skiprows=1, sep=';', encoding='latin-1')
CANCELADOS = pd.read_excel('./INPUT/INFO_MATRICULAS.xlsx', engine='openpyxl', sheet_name='4. CANCELADOS')['CODIGO_UIS'].unique()
CODIGOS = DATOS['ESTUDIANTE'].unique()
CICLO = 1
ASIGNATURA_PESADA = 'PYTHON'
PONDERADO_PESADA = 0.8
PONDERADO_LIVIANAS = 0.1
UMBRAL_REPROBADO = 2.945
REDONDEO_DECIMALES = 1

In [3]:
rta = {
    'CODIGO': [],
    'NOMBRE': [],
    'APELLIDO': [],
    'DEFINITIVA_REAL': [],
    'DEFINITIVA_CICLO': [],
    'REPROBÓ': [],
    'CANCELÓ_MATRICULA': []
}
asignaturas = DATOS['ASIGNATURA'].unique()
for asignatura in asignaturas:    
    rta[asignatura] = []
for cod in CODIGOS:
    df = DATOS[DATOS['ESTUDIANTE'] == cod]
    acum = 0
    for index, row in df.iterrows():
        rta[row['ASIGNATURA']].append(row['NOTA'])
        if row['ASIGNATURA'] == ASIGNATURA_PESADA:
            acum += row['NOTA'] * PONDERADO_PESADA
        else:
            acum += row['NOTA'] * PONDERADO_LIVIANAS
    definitiva = round(acum, REDONDEO_DECIMALES)
    rta['CODIGO'].append(cod)
    rta['NOMBRE'].append(row['NOMBRES'])
    rta['APELLIDO'].append(row['APELLIDOS'])
    rta['DEFINITIVA_REAL'].append(acum)
    rta['DEFINITIVA_CICLO'].append(definitiva)
    rta['REPROBÓ'].append(True if definitiva < UMBRAL_REPROBADO else False)
    rta['CANCELÓ_MATRICULA'].append(True if cod in CANCELADOS else False)
rCompleto = pd.DataFrame(rta)
rCompleto = rCompleto[['CODIGO', 'NOMBRE', 'APELLIDO'] + list(asignaturas) + ['DEFINITIVA_REAL', 'DEFINITIVA_CICLO', 'REPROBÓ', 'CANCELÓ_MATRICULA']]
rCompleto.to_excel('./OUTPUT/DEFINITIVAS_CICLO_' + str(CICLO) + '_' + str(date.today()) + '.xlsx', index = False)
rNivelar = rCompleto[(rCompleto['REPROBÓ'] == True) & (rCompleto['CANCELÓ_MATRICULA'] == False)]
rNivelar.to_excel('./OUTPUT/ESTUDIANTES_NIVELAR_CICLO_' + str(CICLO) + '_' + str(date.today()) + '.xlsx', index = False)